# Northwind database analysis

## Executive Summary 

In [1]:
#imports
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import inspect

import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# create the SQL engine
# to use the large version of the databse:
# engine = create_engine('sqlite:///Northwind_large.sqlite', echo=True)

# to use the small version of the databse:
engine = create_engine('sqlite:///Northwind_small.sqlite', echo=True)


Session = sessionmaker(bind=engine)
session = Session()


In [3]:
# have a look at the tables
inspector = inspect(engine)
print(inspector.get_table_names())


2019-04-01 22:51:20,413 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-04-01 22:51:20,416 INFO sqlalchemy.engine.base.Engine ()
2019-04-01 22:51:20,417 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-04-01 22:51:20,419 INFO sqlalchemy.engine.base.Engine ()
2019-04-01 22:51:20,422 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-04-01 22:51:20,424 INFO sqlalchemy.engine.base.Engine ()
['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']


In [4]:
def get_columns_info(col_name):
    cols_list = inspector.get_columns(col_name)
    
    print("Table Name: {}".format(col_name))
    print("")
    
    for column in cols_list:
        print("Name: {} \t Type: {}".format(column['name'], column['type']))

get_columns_info('OrderDetail')




2019-04-01 22:51:21,549 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("OrderDetail")
2019-04-01 22:51:21,551 INFO sqlalchemy.engine.base.Engine ()
Table Name: OrderDetail

Name: Id 	 Type: VARCHAR(8000)
Name: OrderId 	 Type: INTEGER
Name: ProductId 	 Type: INTEGER
Name: UnitPrice 	 Type: DECIMAL
Name: Quantity 	 Type: INTEGER
Name: Discount 	 Type: FLOAT


In [12]:
def get_columns_info(col_name):
    cols_list = inspector.get_columns(col_name)
    
    print("Table Name: {}".format(col_name))
    print("")
    
    for column in cols_list:
        print("Name: {} \t Type: {}".format(column['name'], column['type']))

get_columns_info('Order')

2019-04-01 23:19:58,824 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Order")
2019-04-01 23:19:58,826 INFO sqlalchemy.engine.base.Engine ()
Table Name: Order

Name: Id 	 Type: INTEGER
Name: CustomerId 	 Type: VARCHAR(8000)
Name: EmployeeId 	 Type: INTEGER
Name: OrderDate 	 Type: VARCHAR(8000)
Name: RequiredDate 	 Type: VARCHAR(8000)
Name: ShippedDate 	 Type: VARCHAR(8000)
Name: ShipVia 	 Type: INTEGER
Name: Freight 	 Type: DECIMAL
Name: ShipName 	 Type: VARCHAR(8000)
Name: ShipAddress 	 Type: VARCHAR(8000)
Name: ShipCity 	 Type: VARCHAR(8000)
Name: ShipRegion 	 Type: VARCHAR(8000)
Name: ShipPostalCode 	 Type: VARCHAR(8000)
Name: ShipCountry 	 Type: VARCHAR(8000)


In [5]:
def get_columns_info(col_name):
    cols_list = inspector.get_columns(col_name)
    print(cols_list)
#     print("Table Name: {}".format(col_name))
#     print("")
    
#     for column in cols_list:
#         print("Name: {} \t Type: {}".format(column['name'], column['type']))

get_columns_info('OrderDetail')

[{'name': 'Id', 'type': VARCHAR(length=8000), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'OrderId', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'ProductId', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'UnitPrice', 'type': DECIMAL(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'Quantity', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'Discount', 'type': FLOAT(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}]


In [6]:
con = engine.connect()
rs = con.execute('SELECT * FROM Customer LIMIT 5')

print(rs.fetchall())



2019-04-01 22:51:22,832 INFO sqlalchemy.engine.base.Engine SELECT * FROM Customer LIMIT 5
2019-04-01 22:51:22,834 INFO sqlalchemy.engine.base.Engine ()
[('ALFKI', 'Alfreds Futterkiste', 'Maria Anders', 'Sales Representative', 'Obere Str. 57', 'Berlin', 'Western Europe', '12209', 'Germany', '030-0074321', '030-0076545'), ('ANATR', 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Owner', 'Avda. de la Constitución 2222', 'México D.F.', 'Central America', '05021', 'Mexico', '(5) 555-4729', '(5) 555-3745'), ('ANTON', 'Antonio Moreno Taquería', 'Antonio Moreno', 'Owner', 'Mataderos  2312', 'México D.F.', 'Central America', '05023', 'Mexico', '(5) 555-3932', None), ('AROUT', 'Around the Horn', 'Thomas Hardy', 'Sales Representative', '120 Hanover Sq.', 'London', 'British Isles', 'WA1 1DP', 'UK', '(171) 555-7788', '(171) 555-6750'), ('BERGS', 'Berglunds snabbköp', 'Christina Berglund', 'Order Administrator', 'Berguvsvägen  8', 'Luleå', 'Northern Europe', 'S-958 22', 'Sweden', '0921-12 34 

In [7]:
rs = con.execute('SELECT firstname, lastname, title FROM Employee')
df = pd.DataFrame(rs.fetchall())

df.head()



2019-04-01 22:51:27,015 INFO sqlalchemy.engine.base.Engine SELECT firstname, lastname, title FROM Employee
2019-04-01 22:51:27,018 INFO sqlalchemy.engine.base.Engine ()


,0,1,2
0,Nancy,Davolio,Sales Representative
1,Andrew,Fuller,"Vice President, Sales"
2,Janet,Leverling,Sales Representative
3,Margaret,Peacock,Sales Representative
4,Steven,Buchanan,Sales Manager


In [14]:
rs = con.execute('SELECT EmployeeId FROM Order')
df = pd.DataFrame(rs.fetchall())

df.head()

2019-04-01 23:25:26,772 INFO sqlalchemy.engine.base.Engine SELECT EmployeeId FROM Order
2019-04-01 23:25:26,774 INFO sqlalchemy.engine.base.Engine ()
2019-04-01 23:25:26,776 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) near "Order": syntax error [SQL: 'SELECT EmployeeId FROM Order']

In [8]:
def make_column_list(table):
    cols = []  #make an empty list
    cols_dict = inspector.get_columns(table)  # make a dict of the column names from the SQL table
#     print(cols_list)
    for column in cols_dict:
        cols.append(column['name'])
#     print(cols)
    return cols


In [9]:
# small sample of orders for testing
# rs = con.execute('SELECT * FROM OrderDetail LIMIT 25')

# get full orderdetail
rs = con.execute('SELECT * FROM OrderDetail')
order_detail = pd.DataFrame(rs.fetchall())

column_names = make_column_list('OrderDetail')
order_detail.columns = column_names

order_detail.head()

2019-04-01 22:51:33,795 INFO sqlalchemy.engine.base.Engine SELECT * FROM OrderDetail
2019-04-01 22:51:33,798 INFO sqlalchemy.engine.base.Engine ()


,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0
2,10248/72,10248,72,34.8,5,0.0
3,10249/14,10249,14,18.6,9,0.0
4,10249/51,10249,51,42.4,40,0.0


In [11]:
# get full orders
rs = con.execute('SELECT * FROM Order LIMIT 5')

print(rs.fetchall())


# order = pd.DataFrame(rs.fetchall())

# column_names = make_column_list('Order')
# order.columns = column_names

# order.head()

2019-04-01 23:01:44,568 INFO sqlalchemy.engine.base.Engine SELECT * FROM Order LIMIT 5
2019-04-01 23:01:44,570 INFO sqlalchemy.engine.base.Engine ()
2019-04-01 23:01:44,572 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) near "Order": syntax error [SQL: 'SELECT * FROM Order LIMIT 5']

In [ ]:
# get full customer
rs = con.execute('SELECT * FROM Customer')
customer = pd.DataFrame(rs.fetchall())

column_names = make_column_list('Customer')
customer.columns = column_names

customer.head()

In [ ]:
plt.hist(order_detail["Discount"])
plt.show()

In [ ]:
# make a new column to calculate the line item total
order_detail['line_total'] = order_detail['Quantity'] * order_detail['UnitPrice'] * (1 - order_detail['Discount'])

In [ ]:
order_detail.head()

In [ ]:
order_detail.describe()

In [ ]:
plt.hist(order_detail['line_total'])

In [ ]:
plt.scatter(order_detail['Discount'], order_detail['line_total'])

Idea - Make a pandas dataframe of all orders where there is any item with a discount. 
Within those orders, what are the summary statistics for:
* The number of different SKU's in the order 
* Quantity of each SKU ordered. How does this compare to orders where the samei SKU has been ordered but there was no discount.
* 

And then compare those statistics to orders where there was no item with a discount.










In [ ]:
plt.scatter(order_detail['Discount'], order_detail['Quantity'])